In [1]:
from Arena import Arena
from BoopPlayers import *
from BoopGame import BoopGame
from BoopLogic import Board

g = BoopGame(6)

agentRandom = RandomPlayer(g).play
agentRandomWithCenterBias = RandomPlayerWithCenterBias(g).play

arena = Arena(agentRandom, agentRandomWithCenterBias, g, display=BoopGame.display)

numOfGames = 10
oneWon, twoWon, iterations = arena.playGames(numOfGames, verbose=True)

print("Player 1's Wins: ", oneWon)
print("Player 2's Wins: ", twoWon)
print("Average Number of Turns in a Game: ", iterations/numOfGames)

Arena.playGames (1):   0%|                                                                       | 0/5 [00:00<?, ?it/s]

Turn  1 Player  1


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

___________________
Turn  2 Player  -1


array([[0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

...

array([[ 2,  0,  0,  0,  0,  2],
       [ 0,  0,  0, -2,  0, -2],
       [-2,  0,  2,  0,  2,  0],
       [ 0,  2,  0,  0, -2, -1],
       [ 0,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2]])

___________________
Game over: Turn  79 Result  -1


array([[ 2,  0,  0,  0,  0,  2],
       [ 0,  0,  0, -2,  0, -2],
       [-2,  0,  2,  0,  2,  0],
       [ 0,  2,  0,  0, -2, -1],
       [ 0,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  2]])

Arena.playGames (2): 100%|███████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.84it/s]

Player 1's Wins:  3
Player 2's Wins:  7
Average Number of Turns in a Game:  63.0
